# Bac à sable

## Check header CSV source

In [77]:
import os.path
import pandas as pd
import numpy

### Paramètres 

In [89]:
UPLOAD_FOLDER = 'uploads'
DOWNLOAD_FOLDER = 'downloads'
EXPORT_FILENAME = 'points.scr'
ALLOWED_EXTENSIONS = {'profil': 'csv','lidar': 'csv','dxf2kml': 'dxf', 'vc_lidar':'csv'}
SOURCE_FILE = '../static/data_sample/LIDAR_profile.csv'
CSV_COLUMS = {'lidar':'distance,altitude,color_packed,intensity,classification,coords',
              'profil':'distance,bathymetry,mns,mnt,x,y',
              'vc_lidar':'distance,altitude,color_packed,intensity,classification,coords'}
LIDAR_CLASSIFICATION_IGNORE = [4,5,6]

In [3]:
print(type(CSV_COLUMS['profil']))

<class 'str'>


### Fonctions de contrôle

In [46]:
def allowed_file(filename:str, conversion: str) -> bool:
    extension = get_file_extension(filename)
    return True if extension == ALLOWED_EXTENSIONS[conversion] else False

def get_file_name(filepath: str) -> str:
    return filepath.split('/')[-1]

def get_file_extension(filename: str) -> str:
    return filename.rsplit('.', 1)[1].lower()

def get_first_line(file: str) -> list:
    with open(file, encoding='utf-8-sig') as f:
        first_line = f.readline().strip('\n')
    first_line = first_line.replace('"', '')
    return first_line.split(",")

# -------------------

def check_valid_header_csv(file: str, conversion: str) -> bool:
    src_header =get_first_line(file)
    request_header = CSV_COLUMS[conversion].split(",")
    return True if src_header == request_header else False


def check_file_extension(file: str, extension: str) -> bool:
    return True if file.rsplit('.', 1)[1].lower() == extension else False

def check_valid_csv_header(file: str, convertion: str) -> bool:
    with open(file, encoding='utf-8-sig') as f:
        first_line = f.readline().strip('\n')

    a = first_line.replace('"', '').split(",")
    b = tuple(CSV_COLUMS[convertion])
    return True if a == b else False





def csv_header_check(df):
    source_colums = get_csv_header(df)
    if source_colums == CSV_COLUMS['profil'] or source_colums == CSV_COLUMS['lidar']:
        return True
    else:
        return False

def get_csv_header(df):
    return df.columns

def get_csv_type_profile(df):
    if csv_header_check(df) :
        source_colums = get_csv_header(df)
        if source_colums == CSV_COLUMS['profil']:
            return 'profile'
        else:
            return 'lidar'
    else:
        return


### Tests des fonctions

#### Nom du fichier et extention

In [60]:
file = SOURCE_FILE

print(get_file_name(SOURCE_FILE))
print(get_file_extension(SOURCE_FILE))
filename = get_file_name(SOURCE_FILE)
print(filename)
print(get_file_extension(filename))

LIDAR_profile.csv
csv
LIDAR_profile.csv
csv


### Retourne l'extension attendu pour chaque type de conversion

In [12]:
print(ALLOWED_EXTENSIONS['profil'])
print(ALLOWED_EXTENSIONS['lidar'])
print(ALLOWED_EXTENSIONS['dxf2kml'])
print(ALLOWED_EXTENSIONS['vc_lidar'])

csv
txt
dxf
csv


### Contrôle si l'extenmsion est celle attendue pour la conversion

In [61]:
print(allowed_file(filename, 'lidar'))

True


### Contrôle de l'extension selon la conversion

In [62]:
if allowed_file(filename, 'profil'):
    print("File type OK")
else:
    print("File type error")

File type OK


### Lire la première line du fichier

In [63]:
print(get_first_line(SOURCE_FILE))
print(type(get_first_line(SOURCE_FILE)))

['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']
<class 'list'>


### Contrôle si l'en-tête est celle attendue

In [64]:
check_valid_header_csv(file, 'lidar')
# print(check_valid_csv_header(file,CSV_COLUMS['profil']))

True

#### Chargement du fichier CSV dans le dataframe

In [65]:
df = pd.read_csv(file)

#### Récupère les noms de colonnes dans une liste

In [66]:
print(df.columns.to_list())

['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']


#### Affiche les valeurs attendues pour profil et lidar

In [56]:
print(CSV_COLUMS['lidar'].split(","))
print(type(CSV_COLUMS['lidar'].split(",")))
print(CSV_COLUMS['profil'].split(","))
print(CSV_COLUMS['vc_lidar'].split(","))

['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']
<class 'list'>
['distance', 'bathymetry', 'mns', 'mnt', 'x', 'y']
['distance', 'altitude', 'color_packed', 'intensity', 'classification', 'coords']


### Nom fichier de sortie

In [67]:
print(filename.split(".")[0] + ".scr")

LIDAR_profile.scr


### Créer un dataframe avec les colonnes nécessaires

In [101]:
df = df[['distance', 'altitude', 'classification']]
print(df)


      distance  altitude  classification
0         0.25   458.153              31
1         0.25   458.154              31
2         0.26   458.041               2
3         0.43   458.123               2
4         1.92   458.078               2
...        ...       ...             ...
2257    418.04   430.447               4
2258    418.16   431.824               4
2259    418.42   432.178               4
2260    418.96   429.937               4
2261    420.14   429.300              41

[2262 rows x 3 columns]


### Créer des groupes par classification

In [102]:
gb = df.groupby('classification')
print(type(pd.unique(df['classification'])[0]))

for g in gb:
    print(g)

<class 'numpy.int64'>
(2,       distance  altitude  classification
2         0.26   458.041               2
3         0.43   458.123               2
4         1.92   458.078               2
5         1.92   458.079               2
6         2.37   458.157               2
...        ...       ...             ...
2146    385.54   429.130               2
2151    386.28   429.369               2
2152    386.28   429.369               2
2213    406.62   429.121               2
2214    406.62   429.121               2

[379 rows x 3 columns])
(3,       distance  altitude  classification
274      84.46   460.507               3
356      90.85   460.308               3
369      92.90   460.171               3
370      92.90   460.171               3
581     123.73   459.457               3
...        ...       ...             ...
2242    413.73   429.409               3
2245    414.18   429.669               3
2246    414.84   429.610               3
2247    415.45   429.283               3
22

### Liste les classifications à ignorer

In [103]:
print(LIDAR_CLASSIFICATION_IGNORE)
print(type(LIDAR_CLASSIFICATION_IGNORE))

[4, 5, 6]
<class 'list'>


### Filtre le dataframe sans les classifaction à ignorer

In [104]:
df_filtred = df[~df.classification.isin(LIDAR_CLASSIFICATION_IGNORE)]
print(df_filtred)


      distance  altitude  classification
0         0.25   458.153              31
1         0.25   458.154              31
2         0.26   458.041               2
3         0.43   458.123               2
4         1.92   458.078               2
...        ...       ...             ...
2248    415.45   429.283               3
2251    415.55   429.167              18
2252    415.55   429.168              18
2255    416.92   429.173               9
2261    420.14   429.300              41

[1113 rows x 3 columns]
